In [337]:
import pandas as pd
import numpy as np
import seaborn as sbn

# Carga de datos 

In [338]:
df_products = pd.read_csv("../dataset_amazon/amazon - amazon_product.csv")
print(f"Filas y columnas de productos dataset: {df_products.shape}")

df_reviews = pd.read_csv("../dataset_amazon/amazon - amazon_review.csv")
print(f"Filas y columnas de reviews dataset: {df_reviews.shape}")

Filas y columnas de productos dataset: (1469, 7)
Filas y columnas de reviews dataset: (1465, 10)


# Tipos de datos y valores no nulos por columna

In [339]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1469 entries, 0 to 1468
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1469 non-null   object
 1   product_name         1469 non-null   object
 2   category             1469 non-null   object
 3   discounted_price     1469 non-null   object
 4   actual_price         1469 non-null   object
 5   discount_percentage  1469 non-null   object
 6   about_product        1465 non-null   object
dtypes: object(7)
memory usage: 80.5+ KB


In [340]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         1465 non-null   object
 1   user_name       1465 non-null   object
 2   review_id       1465 non-null   object
 3   review_title    1465 non-null   object
 4   review_content  1465 non-null   object
 5   img_link        999 non-null    object
 6   product_link    999 non-null    object
 7   product_id      1465 non-null   object
 8   rating          1465 non-null   object
 9   rating_count    1463 non-null   object
dtypes: object(10)
memory usage: 114.6+ KB


# Identificación de valores nulos

Para identificar qué columnas tienen nulos se usó *isnull().sum()* , para el caso de *products* se encontró que en la columna **about_product** hay 4 nulos, se tomará la decisión de imputar con *"sin info"* más adelante.

In [341]:
print(f" Nulos en productos: \n {df_products.isnull().sum()}")

 Nulos en productos: 
 product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
about_product          4
dtype: int64


De igual manera para *reviews* se identificaron valores nulos, en este caso **img_links** y **product_link** tienen 466 valores nulos, sin embargo, considerando las preguntas a responder en este análisis, esas columnas no nos aportan información relevante, por lo que se sugiere eliminar esas columnas. Por otro lado para **rating_count** hay 2 valores nulos, en este caso, se plantea eliminar esos valores, ya que son un bajo porcentaje y así no sesgamos los datos suponiendo que nadie lo valoró.

In [342]:
print(f" Nulos en reviews: \n  {df_reviews.isnull().sum()}")

 Nulos en reviews: 
  user_id             0
user_name           0
review_id           0
review_title        0
review_content      0
img_link          466
product_link      466
product_id          0
rating              0
rating_count        2
dtype: int64


# Manejo de valores nulos

### DataFrame *products* 

```
df_products['about_product'] → selecciona la columna about_product del DataFrame df_products.
.fillna("Sin descripción") → reemplaza todos los valores nulos (NaN) de esa columna con el texto "Sin descripción".
El resultado se reasigna a la misma columna.
```

In [343]:
df_products['about_product'] = df_products['about_product'].fillna("Sin descripción")

### DataFrame *reviews* 

```
.drop(columns=[...]) → elimina columnas completas del DataFrame. Aquí borramos img_link y product_link.
Se reasigna a df_reviews para guardar el DataFrame ya sin esas columnas.
```

```
.dropna(...) → elimina filas que contienen valores nulos.
subset=['rating_count'] → le decimos que solo revise la columna rating_count. Si en esa columna encuentra NaN, elimina esa fila completa.
Se reasigna a df_reviews para guardar los cambios.
```

In [344]:
df_reviews = df_reviews.drop(columns=['img_link', 'product_link'])
df_reviews = df_reviews.dropna(subset=['rating_count'])

Comprobamos que ya no tenemos nulos

In [345]:
print(f" Nulos en productos: \n {df_products.isnull().sum()}")
print(f" Nulos en reviews: \n  {df_reviews.isnull().sum()}")

 Nulos en productos: 
 product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
about_product          0
dtype: int64
 Nulos en reviews: 
  user_id           0
user_name         0
review_id         0
review_title      0
review_content    0
product_id        0
rating            0
rating_count      0
dtype: int64


# Normalizamos valores en DataFrame **products**

In [346]:
# Precios
df_products['discounted_price'] = df_products['discounted_price'].str.replace('₹','').str.replace(',','').astype(float)
df_products['actual_price'] = df_products['actual_price'].str.replace('₹','').str.replace(',','').astype(float)

# Descuento
df_products['discount_percentage'] = df_products['discount_percentage'].str.replace('%','').astype(float)


# Valores duplicados

Contamos duplicados en **products**, usando *duplicated().sum()* , nos dice cuántas columnas completas están duplicadas, es decir, todas las columnas iguales.

In [347]:
print(df_products.duplicated().sum())

106


De igual manera contamos duplicados en *reviews* 

In [348]:
print(df_reviews.duplicated().sum())

70


En el caso de **products** hay 106 filas iguales, y en **reviews** hay 70 filas iguales, por lo que hay que eliminar filas idénticas para cada caso.

# Eliminación de filas idénticas

In [349]:
#products
df_products = df_products.drop_duplicates()
#reviews
df_reviews = df_reviews.drop_duplicates()


Revisamos **id's** , es decir en products y reviews las columnas de **product_id** y de **review_id**, ya que se espera que sean identificadores únicos.

In [350]:
print(df_products['product_id'].duplicated().sum())
print(df_reviews['review_id'].duplicated().sum())

12
201


En el caso de *products*, aquellos que había imputado con "Sin descripción" resultaron ser repetidos, por lo que ahora sí eliminaremos esos valores de **about_product** : 

In [351]:
# Eliminar productos cuya descripción es "Sin descripción"
df_products = df_products[df_products['about_product'] != "Sin descripción"]

In [352]:
print(df_products['product_id'].duplicated().sum())
print(df_reviews['review_id'].duplicated().sum())

8
201


Aún se observaron 8 productos con id duplicado, y 201 en reviews con mismo id.

In [353]:
# IDs de productos repetidos
dup_products = df_products[df_products['product_id'].duplicated(keep=False)]
print(dup_products.sort_values(by='product_id'))



     product_id                                       product_name  \
433  B07DJLFMPS  HP 32GB Class 10 MicroSD Memory Card (U1 TF Ca...   
686  B07DJLFMPS  HP 32GB Class 10 MicroSD Memory Card (U1 TF Ca...   
15   B083342NKJ  MI Braided USB Type-C Cable for Charging Adapt...   
699  B083342NKJ  MI Braided USB Type-C Cable for Charging Adapt...   
10   B08CF3D7QR  Portronics Konnect L POR-1081 Fast Charging 3A...   
428  B08CF3D7QR  Portronics Konnect L POR-1081 Fast Charging 3A...   
2    B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
379  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
623  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
397  B09MT84WV5  Samsung EVO Plus 128GB microSDXC UHS-I U3 130M...   
641  B09MT84WV5  Samsung EVO Plus 128GB microSDXC UHS-I U3 130M...   
336  B0B5B6PQCT  boAt Wave Call Smart Watch, Smart Talk with Ad...   
587  B0B5B6PQCT  boAt Wave Call Smart Watch, Smart Talk with Ad...   
344  B0B5LVS732  Noi

Para el caso de **products** decidí quedarme con aquellos que tenían un descuento menor, para considerar "el peor caso".

```
df_products.sort_values('discounted_price') Ordena el DataFrame de menor a mayor según discounted_price.Es decir, los precios más bajos aparecen primero.

.drop_duplicates(subset='product_id', keep='first') Busca duplicados en la columna product_id.

Para cada product_id repetido, mantiene la primera fila (la que tiene el menor discounted_price, porque ya ordenaste).
```

In [ ]:
df_products = df_products.sort_values('discounted_price').drop_duplicates(subset='product_id', keep='first')

In [358]:
# Ver cuántos product_id están duplicados
print(f"Ver product_id duplicados: {df_products['product_id'].duplicated().sum()}")  # debe dar 0

# Ver cuántos product_id únicos hay vs total de filas
print(df_products['product_id'].nunique(), "product_id únicos")
print(len(df_products), "filas en total")


Ver product_id duplicados: 0
1351 product_id únicos
1351 filas en total


## Reviews

In [356]:
# IDs de reseñas repetidas
dup_reviews = df_reviews[df_reviews['review_id'].duplicated(keep=False)]
print(dup_reviews.sort_values(by='review_id'))

                                               user_id  \
84   AEXK37TSBFHSP2TYE63YPKETWQ7Q,AEKMVX2VDNNX4ZFXI...   
261  AEXK37TSBFHSP2TYE63YPKETWQ7Q,AEKMVX2VDNNX4ZFXI...   
523  AFTS5BKDRY7Y23B27UVBE2V6TOHA,AHRIDJXYEBQS7MXFD...   
466  AFTS5BKDRY7Y23B27UVBE2V6TOHA,AHRIDJXYEBQS7MXFD...   
672  AFLBLMPC4WUEDUWHLHBQVY5AKH2A,AE4ZXGSA2CQOGKH3N...   
..                                                 ...   
463  AH7LW3BCJBLCZTMWBOFL33UGIRBQ,AFSJYBGBY2U6KAAUR...   
506  AH7LW3BCJBLCZTMWBOFL33UGIRBQ,AFSJYBGBY2U6KAAUR...   
492  AH7LW3BCJBLCZTMWBOFL33UGIRBQ,AFSJYBGBY2U6KAAUR...   
62   AF42EMTPEJAL4LNEPPX77TN77UHA,AHBMZRY43T2GTYDVN...   
118  AF42EMTPEJAL4LNEPPX77TN77UHA,AHBMZRY43T2GTYDVN...   

                                             user_name  \
84   Sunil Funde,Biju Abraham Thomas,Samir,Rahul Sh...   
261  Sunil Funde,Biju Abraham Thomas,Samir,Rahul Sh...   
523  Ranit Barman,Ravi Singh,Karan Rai,Amazon Custo...   
466  Ranit Barman,Ravi Singh,Karan Rai,Amazon Custo...   
672  Velir,Di